In [141]:
# goal
# step1. given (a) vine structure, (b) copula family and  (c) copula parameters
# step2. generate data
# step3. mask var5 with x% dropout
# step4. infer the (a-c) on masked data using complete cases
# step5.1 if the vine structure supports direct imputation of var5 -> admit to comparison
# step5.2 if the vine structure does not support direct imputation of var5 -> skip
# step6. run zhao/udell's code to impute masked data
# step7. compute smae imputation error, bias of correlation from step 5.1,6

# reference
# the pair-copula index for edge e in tree t of a d dimensional vine is
# (M[d - 1 - e, e], M[t, e]; M[t - 1, e], ..., M[0, e])

In [142]:
import numpy as np
from matplotlib import pyplot as plt
import pyvinecopulib as pv

def get_ced_cing(T, cop=None):
    d = T.shape[1]
    cing = []
    ced = []
    param = []
    for j in range(d):
        for i1 in range(d-j-1):
            ced.append(sorted((T[i1,j], T[d-j-1, j])))
            tmp = []
            for i2 in range(i1):
                tmp.append(T[i2,j])
            cing.append(sorted(tmp))
            if cop is not None:
                param.append(cop.get_parameters(i1, j)[0][0])
    return ced, cing, param

def find(a, cing_len, ced, cing):
    out = [i for i in range(len(ced)) if a in ced[i]]
    matched = False
    for i in out:
        if len(cing[i]) == cing_len:
            matched = True
            break
    assert matched, f'bad argument, a={a}, cing_len={cing_len}'
    return ced[i][1] if ced[i][0] == a else ced[i][0]

def diagonalize(T1, a):
    d = T1.shape[1]
    if a == T1[d-1, 0]:
        return T1
    assert a == T1[d-2, 0], f'cannot be diagonalized with {a}'

    T2 = np.zeros(shape=T1.shape, dtype=np.uint64)
    T2[d-1, 0] = a
    order = [a]

    ced, cing, _ = get_ced_cing(T1)
    for j in range(d-1):
        for i in range(d-j-1):
            T2[i,j] = find(T2[d-j-1, j], i, ced, cing)

        remove_idx = [i for i, c in enumerate(ced) for k in order if k in c]
        keep_idx = set(range(len(ced))).difference(set(remove_idx))

        ced = [ced[i] for i in keep_idx]
        cing = [cing[i] for i in keep_idx]

        T2[d-j-2, j+1] = T2[d-j-2, j]
        order.append(T2[d-j-2, j+1])
    return T2

def make_diagonal_copula(cop1, a):
    T1 = cop1.matrix
    d = T1.shape[0]
    ced, cing, param = get_ced_cing(T1, cop1)

    T2 = diagonalize(T1, a)

    pair_copulas = []
    for t in range(d-1):
        cur = []
        pair_copulas.append(cur)
        for e in range(d-1-t):
            cur.append(
                pv.Bicop(
                    family=pv.BicopFamily.gaussian,
                    parameters=[param[ced.index(sorted((T2[d-1-e,e], T2[t,e])))]]
                )
            )
    cop2 = pv.Vinecop(matrix=T2, pair_copulas=pair_copulas)
    return cop2

In [143]:
d = 5
corr = np.array([
    [0.5, 0.25, 0.25, 0.9], 
    [0.1, 0.9, 0.9, np.nan], 
    [0.125, 0.66, np.nan, np.nan],
    [0.55, np.nan, np.nan, np.nan]]
)
vine_matrix = np.array([
    [3,2,3,3,3],
    [2,3,2,2,0],
    [4,4,4,0,0],
    [1,1,0,0,0],
    [5,0,0,0,0]
])

In [144]:
pair_copulas = []
for t in range(d-1):
    cur = []
    pair_copulas.append(cur)
    for e in range(d-1-t):
        cur.append(
            pv.Bicop(
                family=pv.BicopFamily.gaussian,
                parameters=[corr[t,e]]
            )
        )
cop = pv.Vinecop(matrix=vine_matrix, pair_copulas=pair_copulas)
u = cop.simulate(1000, seeds=[1,2,3,4,5])

In [145]:
missing = np.random.binomial(n=1, p=0.2, size=u.shape[0])
present = (1 - missing)
u_mask = np.copy(u)
u_mask[:, 4] = np.where(missing > 0, np.nan, u_mask[:,4])

In [146]:
fit_controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian])
cop1 = pv.Vinecop(u_mask, controls=fit_controls)
cop2 = make_diagonal_copula(cop1, 5)

In [148]:
cop2.matrix

array([[3, 2, 3, 3, 3],
       [1, 3, 2, 2, 0],
       [2, 1, 1, 0, 0],
       [4, 4, 0, 0, 0],
       [5, 0, 0, 0, 0]], dtype=uint64)

In [149]:
cop2

<pyvinecopulib.Vinecop>
** Tree: 0
5,3 <-> Gaussian, parameters = 0.478123
4,2 <-> Gaussian, parameters = 0.625168
1,3 <-> Gaussian, parameters = 0.571803
2,3 <-> Gaussian, parameters = 0.900277
** Tree: 1
5,1 | 3 <-> Gaussian, parameters = 0.218916
4,3 | 2 <-> Gaussian, parameters = -0.844873
1,2 | 3 <-> Gaussian, parameters = -0.855547
** Tree: 2
5,2 | 1,3 <-> Gaussian, parameters = 0.459573
4,1 | 3,2 <-> Gaussian, parameters = 0.663084
** Tree: 3
5,4 | 2,1,3 <-> Gaussian, parameters = -0.302255

In [159]:
u1 = u_mask[:, 0][:, None]
u2 = u_mask[:, 1][:, None]
u3 = u_mask[:, 2][:, None]
u4 = u_mask[:, 3][:, None]
u5_true = u[:, 4][:, None]

imputed = None
for i in range(10):
    v = np.random.uniform(size=u_mask.shape[0])[:, None]
    u5_4 = cop2.get_pair_copula(3,0).hinv1(np.hstack([v, u4]))[:, None]
    u5_42 = cop2.get_pair_copula(2,0).hinv1(np.hstack([u5_4, u2]))[:, None] # fix this
    u5_421 = cop2.get_pair_copula(1,0).hinv1(np.hstack([u5_42, u1]))[:, None]
    u5_4213 = cop2.get_pair_copula(0,0).hinv1(np.hstack([u5_421, u3]))[:, None]
    if imputed is None:
        imputed = u5_4213
    else:
        imputed = (imputed + u5_4213)/2

In [ ]:
cop2.get_pair_copula

In [166]:
np.corrcoef(np.ravel(u5_true), np.ravel(u2))

array([[1.        , 0.41046857],
       [0.41046857, 1.        ]])

In [168]:
np.corrcoef(np.ravel(u5_42), np.ravel(u2))

array([[1.        , 0.94772115],
       [0.94772115, 1.        ]])

In [163]:
numer = imputed - u5_true
denom = np.nanmedian(u_mask[:,4]) - u5_true
smae = np.linalg.norm(numer[missing == 1], ord=1) / np.linalg.norm(denom[missing == 1], ord=1)
print(smae)

1.0346233277907437
